### START

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, recall_score, precision_score, classification_report

/home/metaphysicist/Coding/FORWARD/Mastering/.nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("CUDA availaible:", torch.cuda.is_available())
print("Current device index:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA availaible: True
Current device index: 0
Device name: NVIDIA GeForce GTX 1650 with Max-Q Design


### DATA PREPARION

In [3]:
def read_conli_file(file_path):
    with open(file_path, "r") as f:
        content = f.read().strip()
        sentences = content.split("\n\n")
        data = []
        for sentence in sentences:
            tokens = sentence.split("\n")
            token_data = []
            for token in tokens:
                token_data.append(token.split())
            data.append(token_data)
    return data


In [4]:

def convert_to_dataset(data,label_map):
    formatted_data = {"tokens":[],"ner_tags":[]}
    for sentence in data:
        tokens = [token_data[0] for token_data in sentence]
        ner_tags = [label_map[token_data[3]] for token_data in sentence]
        formatted_data["tokens"].append(tokens)
        formatted_data["ner_tags"].append(ner_tags)
    return Dataset.from_dict(formatted_data)


In [5]:

train_data = read_conli_file("conll2003/eng.train")
valid_data = read_conli_file("conll2003/eng.testa")
test_data = read_conli_file("conll2003/eng.testb")



In [6]:
label_list = sorted(list(set([token_data[3] for sentence in train_data for token_data in sentence])))
label_map = {label: i for i, label in enumerate(label_list)}

train_data = convert_to_dataset(train_data,label_map)
test_data = convert_to_dataset(test_data,label_map)
valid_data = convert_to_dataset(valid_data,label_map)

datasets = DatasetDict({
    "train":train_data,
    "test":test_data,
    "validation":valid_data
})

### INIT TOKENIZER & MODEL

In [7]:
print(len(datasets["validation"]))

3466


In [8]:
model_name = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))
    

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

### METRICS


In [9]:
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "classification_report": classification_report(true_predictions, true_labels),
    }


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True, is_split_into_words=True, padding=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=100,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

Map: 100%|██████████| 3466/3466 [00:00<00:00, 14340.39 examples/s]
/home/metaphysicist/Coding/FORWARD/Mastering/.nlp/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Trainer

In [11]:
def data_collator(data):
    input_ids = [torch.tensor(item["input_ids"]) for item in data]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in data]
    labels = [torch.tensor(item["labels"]) for item in data]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_ids":input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,

)

In [12]:
trainer.train()

  5%|▌         | 100/1874 [00:50<11:55,  2.48it/s]

{'loss': 0.3357, 'grad_norm': 1.6143169403076172, 'learning_rate': 4.7331910352187837e-05, 'epoch': 0.05}


 11%|█         | 200/1874 [01:36<13:50,  2.01it/s]

{'loss': 0.1206, 'grad_norm': 7.7172064781188965, 'learning_rate': 4.466382070437567e-05, 'epoch': 0.11}


 16%|█▌        | 300/1874 [02:22<11:33,  2.27it/s]

...

 96%|█████████▌| 1800/1874 [17:26<00:28,  2.58it/s]

{'loss': 0.046, 'grad_norm': 3.5463814735412598, 'learning_rate': 1.9743863393810032e-06, 'epoch': 0.96}


100%|██████████| 1874/1874 [18:03<00:00,  1.73it/s]

{'train_runtime': 1083.9852, 'train_samples_per_second': 13.826, 'train_steps_per_second': 1.729, 'train_loss': 0.07341841547059148, 'epoch': 1.0}


TrainOutput(global_step=1874, training_loss=0.07341841547059148, metrics={'train_runtime': 1083.9852, 'train_samples_per_second': 13.826, 'train_steps_per_second': 1.729, 'total_flos': 960565719981294.0, 'train_loss': 0.07341841547059148, 'epoch': 1.0})

### TEST

In [28]:
sentence = """Rave George Washington was born in 1732 and died in 1799. Berlin Aristotle Emanuel Kant Nietzche"""


tokenized_input = tokenizer(sentence, return_tensors="pt").to(model.device)

print(tokenized_input)
outputs = model(**tokenized_input)


predicted_labels = outputs.logits.argmax(-1)[0]


named_entities = [tokenizer.decode([token]) for token, label in zip(tokenized_input["input_ids"][0], predicted_labels) if label != 0 and label != label_map['O']]


print("Named Entities - Example 1:", named_entities)


{'input_ids': tensor([[  101, 16890,  2707,  1667,  1994,  1108,  1255,  1107, 21121,  1477,
          1105,  1452,  1107, 14287,   119,  3206, 18727, 23271, 14812,  2227,
         27453,  2105,  1584,  4386,   102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}
Named Entities - Example 1: ['Ra', '##ve', 'George', 'Washington', 'Aristotle', 'Emanuel', 'Ka', '##nt', 'Ni', '##et', '##z', '##che']


In [27]:
trainer.save_model("model/")